In [4]:
from pyspark.sql import SparkSession
from dotenv import load_dotenv
from sedona.spark import SedonaContext
import os
from pyspark.sql.functions import col, explode
from datetime import datetime, date
from pyspark import StorageLevel
from pyspark.sql.types import StructType, StructField, StringType, LongType, ArrayType, MapType
from tqdm import tqdm
from sedona.sql.types import GeometryType
from pathlib import Path    
load_dotenv()

True

In [5]:
sedona = SedonaContext.builder() \
    .appName("BDCCFinalExam") \
    .master("local[*]") \
    .config("spark.driver.memory", "7g") \
    .config("spark.executor.memory", "7g") \
    .config("spark.storage.memoryFraction","0.4") \
    .config("spark.memory.fraction","0.6") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
    .config("spark.hadoop.fs.s3a.access.key", os.getenv("AWS_ACCESS_KEY_ID")) \
    .config("spark.hadoop.fs.s3a.secret.key", os.getenv("AWS_SECRET_ACCESS_KEY")) \
    .config("spark.hadoop.fs.s3a.endpoint", "s3.amazonaws.com") \
    .config("spark.hadoop.fs.s3a.fast.upload", "true") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.executor.extraJavaOptions", "-Dcom.amazonaws.services.s3.enableV4=true") \
    .config("spark.driver.extraJavaOptions", "-Dcom.amazonaws.services.s3.enableV4=true") \
    .config("spark.jars.packages",
            'org.apache.sedona:sedona-spark-shaded-3.3_2.12:1.7.1,'
            "org.apache.hadoop:hadoop-aws:3.2.0,"\
            'org.datasyslab:geotools-wrapper:1.7.1-28.5,'\
            "com.amazonaws:aws-java-sdk-bundle:1.11.375") \
    .getOrCreate()

25/06/04 15:50:21 WARN Utils: Your hostname, PeteCastle.local resolves to a loopback address: 127.0.0.1; using 192.168.45.216 instead (on interface en0)
25/06/04 15:50:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/francismarkcayco/.ivy2/cache
The jars for the packages stored in: /Users/francismarkcayco/.ivy2/jars
org.apache.sedona#sedona-spark-shaded-3.3_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e4e8499d-13b6-4236-83e6-9aaa310a597a;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/anaconda3/envs/bdcc-final-exam/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.sedona#sedona-spark-shaded-3.3_2.12;1.7.1 in central
	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in central
	found org.datasyslab#geotools-wrapper;1.7.1-28.5 in central
:: resolution report :: resolve 137ms :: artifacts dl 157ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.375 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.0 from central in [default]
	org.apache.sedona#sedona-spark-shaded-3.3_2.12;1.7.1 from central in [default]
	org.datasyslab#geotools-wrapper;1.7.1-28.5 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   4   |   0   |   0   |   0   ||   4   |   0   |
	------------------------------------

In [6]:
DATASET_DIR = Path("../outputs")

In [7]:
df = sedona.read.parquet(str(DATASET_DIR / "features"))

In [8]:
df.toPandas()

25/06/04 15:50:26 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,gadm,animal_veterinary_services,art_galleries_exhibition_halls,automotive_repair_maintenance,beauty_personal_care_services,cemeteries_crematoria_memorials,commercial_office_buildings,community_social_service_centers,construction_building_services,educational_institutions,...,pu_storage_eq,coworking_startup_incubators,elderly_care_rehabilitation,pu_power,cultural_heritage_sites,public_squares_pedestrian_zones,bridges_tunnels_overpasses,rs_sari_sari,pu_monitoring,features_quarter
0,Batangas,10.0,101.0,75.0,58.0,12.0,1061.0,139.0,168.0,1314,...,175.0,NaN,1.0,2.0,1.0,NaN,NaN,NaN,NaN,2018-10-01
1,Batangas,10.0,68.0,75.0,59.0,13.0,1058.0,137.0,167.0,1286,...,185.0,NaN,1.0,2.0,1.0,NaN,NaN,NaN,NaN,2018-07-01
2,Batangas,14.0,84.0,104.0,73.0,804.0,1079.0,184.0,254.0,4153,...,171.0,NaN,1.0,2.0,1.0,NaN,NaN,NaN,NaN,2020-07-01
3,Batangas,16.0,82.0,104.0,71.0,804.0,1060.0,163.0,201.0,1795,...,173.0,NaN,1.0,2.0,1.0,NaN,NaN,NaN,NaN,2020-01-01
4,Batangas,9.0,66.0,82.0,56.0,797.0,1059.0,151.0,192.0,1412,...,173.0,NaN,1.0,2.0,1.0,NaN,NaN,NaN,NaN,2019-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1165,Apayao,NaN,NaN,NaN,NaN,3.0,1.0,NaN,1.0,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-10-01
1166,Apayao,NaN,NaN,NaN,NaN,3.0,1.0,NaN,1.0,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-04-01
1167,Apayao,NaN,NaN,NaN,NaN,3.0,1.0,NaN,1.0,18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-01
1168,Apayao,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-10-01
